In [1]:
import sys
import torch
import pickle as pkl
sys.path.append('../../../')

from experiments.supervised.product_manifold.script import main, _plot_clusters, _plot_holonomy_dist, _plot_holonomy_surface
from models.supervised.mlp.model import MLP

models_path = "../../../models/supervised/mlp/saved_models"

In [2]:
size = "underfit"
mode = "moon"
epoch = 199
model = MLP(2,7,2,2)
with open(f'{models_path}/2_wide/mlp_{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)

full_path = f'{models_path}/{size}/mlp_{mode}/model_{epoch}.pth'
model.load_state_dict(torch.load(full_path))
if size == "overfit":
	model.num_layers -= 1
	model.layers = model.layers[:-1]
model.eval()


MLP(
  (layers): ModuleList(
    (0-5): 6 x Layer(
      (act_func): Tanh()
      (linear_map): Linear(in_features=2, out_features=2, bias=True)
    )
    (6): Layer(
      (act_func): Sigmoid()
      (linear_map): Linear(in_features=2, out_features=2, bias=True)
    )
  )
)

In [3]:
wrt = "layer_wise"
N=50
sigma = 0.05
X = torch.tensor(dataset.X).float()
holonomy_manifold, start_loop_vectors, loop_points = main(model, X, N, sigma, 0.9, wrt=wrt, K_MAX=10000)

Clustering took 2.6322848796844482 seconds. Best score: 0.9903528094291687. Best config: {'eps': 0.1619216223418936, 'min_samples': 6}
Finding cycles took 0.29749584197998047 seconds
Computing holonomy group took 0.8020906448364258 seconds. 0.0002406512585767854 seconds per cycle
Finding cycles took 0.19490265846252441 seconds
Computing holonomy group took 0.7408199310302734 seconds. 0.000222268206129695 seconds per cycle
Finding cycles took 0.2003629207611084 seconds
Computing holonomy group took 0.7663149833679199 seconds. 0.00022991748675905188 seconds per cycle


In [4]:
_plot_holonomy_surface(holonomy_manifold, loop_points, X, dataset.y, mode, size, wrt, epoch)
_plot_clusters(holonomy_manifold, loop_points, X, dataset.y, mode, size, wrt, epoch)
_plot_holonomy_dist(holonomy_manifold, mode, size, wrt, epoch)